In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip freeze > requirements.txt


In [ ]:
import nltk
from nltk.corpus import wordnet
import random
nltk.download('omw-1.4')

# Baixe o WordNet em português
nltk.download('wordnet')
nltk.download('omw')

def generate_distractors(answer, num_distractors=3):
    distractors = set()

    # Adiciona a resposta correta como um distrator
    distractors.add(answer)

    # Encontra sinônimos da resposta correta
    for synset in wordnet.synsets(answer, lang='por'):
        for lemma in synset.lemmas(lang='por'):
            distractors.add(lemma.name())

    # Encontra antônimos da resposta correta
    for synset in wordnet.synsets(answer, lang='por'):
        for lemma in synset.lemmas(lang='por'):
            for antonym in lemma.antonyms():
                distractors.add(antonym.name())

    # Remove a resposta correta dos distratores
    distractors.discard(answer)

    # Seleciona aleatoriamente um número especificado de distratores
    distractors = random.sample(distractors, min(num_distractors, len(distractors)))

    return distractors

# Exemplo de uso
answer = "cachorro"
distractors = generate_distractors(answer)
print("Resposta correta:", answer)
print("Distratores:", distractors)

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
#from sentence_embeddings import SentenceEmbeddings

trained_model_path = '/content/drive/MyDrive/t5v1_tcc_2024/model'
trained_tokenizer_path = '/content/drive/MyDrive/t5v1_tcc_2024/tokenizer'


class QuestionGeneration:

    def __init__(self, model_dir=None):
        self.model = T5ForConditionalGeneration.from_pretrained(trained_model_path)
        self.tokenizer = T5Tokenizer.from_pretrained(trained_tokenizer_path)
        self.device = torch.device("cuda")
        self.model = self.model.to(self.device)
        self.model.eval()

    def generate(self, answer: str, context: str):
        input_text = '<answer> %s <context> %s ' % (answer, context)
        encoding = self.tokenizer.encode_plus(
            input_text,
            return_tensors='pt',
            max_length=512
        ).to(self.device)
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        outputs = self.model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            num_beams = 3,
            num_return_sequences = 1,
            max_length = 512
        ).to(self.device)
        question_list = []
        for output in outputs:
            question = self.tokenizer.decode(
                output,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True
            )
            question_list.append({'question': question, 'answer': answer, 'context': context})
        return question_list


QG = QuestionGeneration()



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Pergunta: Qual ramo da fisiologia estuda o funcionamento do sistema respiratório humano?
Resposta: Fisiologia Respiratória
Pergunta: Que órgos compem o sistema respiratório?
Resposta: Pulmões
Pergunta: O que o sistema respiratório faz entre o corpo e o ambiente?
Resposta: Troca Gasosa
Pergunta: O que ocorre a troca gasosa nos pulmes?
Resposta: Alvéolos
Pergunta: Quais so asma, enfisema e bronquite crônica?
Resposta: Distúrbios Respiratórios


In [ ]:
context = '''A Fisiologia Respiratória é o ramo da fisiologia que estuda o funcionamento do sistema respiratório humano. O sistema respiratório é responsável pela troca gasosa entre o corpo e o ambiente, fornecendo oxigênio aos tecidos e removendo dióxido de carbono. Consiste em órgãos como nariz, faringe, laringe, traqueia, brônquios e pulmões.

Durante a inspiração, os músculos respiratórios, incluindo o diafragma e os músculos intercostais, se contraem, expandindo a caixa torácica e diminuindo a pressão dentro dos pulmões. Isso permite que o ar entre nos pulmões, onde ocorre a troca gasosa nos alvéolos pulmonares.

Na expiração, os músculos respiratórios relaxam e a caixa torácica retorna à sua posição de repouso, empurrando o ar rico em dióxido de carbono para fora dos pulmões. O sistema respiratório também desempenha um papel na regulação do pH sanguíneo, controlando os níveis de ácido carbônico no sangue.

Distúrbios respiratórios, como asma, enfisema e bronquite crônica, podem afetar a função pulmonar e prejudicar a troca gasosa. O estudo da fisiologia respiratória é crucial para entender esses distúrbios e desenvolver tratamentos eficazes para melhorar a saúde pulmonar.'''
answer_list = [
    "Fisiologia Respiratória",
    "Pulmões",
    "Troca Gasosa",
    "Alvéolos",
    "Distúrbios Respiratórios"
]



# #SE = SentenceEmbeddings()

for answer in answer_list:
    qa = QG.generate(answer, context)
    print(f'Pergunta: {qa[0].get("question")}\nResposta: {qa[0].get("answer")}')

Pergunta: Qual ramo da fisiologia estuda o funcionamento do sistema respiratório humano?
Resposta: Fisiologia Respiratória
Pergunta: Que órgos compem o sistema respiratório?
Resposta: Pulmões
Pergunta: O que o sistema respiratório faz entre o corpo e o ambiente?
Resposta: Troca Gasosa
Pergunta: O que ocorre a troca gasosa nos pulmes?
Resposta: Alvéolos
Pergunta: Quais so asma, enfisema e bronquite crônica?
Resposta: Distúrbios Respiratórios


In [ ]:
context = '''O Brasil é o maior país da América do Sul e o quinto maior do mundo em área territorial e população. Com uma história rica e diversa, o Brasil é conhecido por sua cultura vibrante, paisagens deslumbrantes e biodiversidade única. Colonizado pelos portugueses no século XVI, o Brasil foi um importante centro colonial e se tornou independente em 1822, proclamando a República em 1889.
Com uma população diversificada composta por uma mistura de povos indígenas, europeus, africanos e asiáticos, o Brasil é conhecido por sua rica herança cultural. A música, a dança e as festividades são elementos essenciais da cultura brasileira, com estilos como samba, bossa nova e frevo ganhando destaque internacionalmente.
Economicamente, o Brasil é uma das maiores economias emergentes do mundo, com setores diversificados que incluem agricultura, indústria, serviços e recursos naturais. No entanto, o país também enfrenta desafios significativos, como desigualdade social, pobreza, corrupção e questões ambientais, que continuam a ser áreas de preocupação e foco para o desenvolvimento sustentável.'''
answer_list = [
    "Brasil",
    "agricultura"
]



# #SE = SentenceEmbeddings()

for answer in answer_list:
    qa = QG.generate(answer, context)
    print(f'Pergunta: {qa[0].get("question")}\nResposta: {qa[0].get("answer")}\nDistratores: {generate_distractors(qa[0].get("answer"))}')

<ipython-input-6-76e17b92f752>:31: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  distractors = random.sample(distractors, min(num_distractors, len(distractors)))


Pergunta: Qual pas é o maior do mundo em área territorial e populaço?
Resposta: Brasil
Distratores: ['ibirapiranga', 'brasil', 'ibirapitá']
Pergunta: Qual é um dos setores diversificados do Brasil?
Resposta: agricultura
Distratores: ['pecuária', 'agronegócio', 'cultivo']
